# Des méthodes des descente de gradient

In [85]:
f(x)=sin(x)
df(x)=cos(x) # admettons qu'on a la dérivée sinon on peut l'approximer avec les éléments finis

df (generic function with 1 method)

In [2]:
function bracket_minimum(f, x=0; s=1e-2, k=2.0) # Cette fonction classique permet de réduire le domaine de recherche du min
    a, ya= x,f(x)
    b, yb= a+s, f(a+s)
    if yb > ya
        a, b = b, a
        ya, yb = yb, ya
        s = -s
    end
    while true
        c, yc = b + s, f(b + s)
        if yc > yb
            return a < c ? (a, c) : (c, a)
        end
        a, ya, b, yb = b, yb, c, yc
        s *= k
    end
end  

bracket_minimum (generic function with 2 methods)

In [24]:
(a,c)=bracket_minimum(f)

(-2.55, -0.6300000000000001)

In [34]:
function line_search(f,x,d) # Cette fonction sera utile pour modifier le pas
    obj=c->f(x+c*d)
    a, b = bracket_minimum(obj)
    c = optimize(obj, a, b)
    m=Optim.minimizer(c)
    return x + a*d
end


line_search (generic function with 1 method)

In [35]:
line_search(f,1,2)

-4.1

In [26]:
i=optimize(f, -0.5,0.5)

Results of Optimization Algorithm
 * Algorithm: Brent's Method
 * Search Interval: [-0.500000, 0.500000]
 * Minimizer: -5.000000e-01
 * Minimum: -4.794255e-01
 * Iterations: 37
 * Convergence: max(|x - x_upper|, |x - x_lower|) <= 2*(1.5e-08*|x|+2.2e-16): true
 * Objective Function Calls: 38

In [33]:
Optim.minimizer(i)

-0.49999998893841135

In [73]:
abstract type DescentMethod end
struct GradientDescent <: DescentMethod
    a
end
init!(M::GradientDescent, f, df, x) = M
function step2(M::GradientDescent, f, df, x)
    a,g= M.a, df(x)
    return x-a*g
end

step2 (generic function with 1 method)

In [67]:
mutable struct ConjugateGradientDescent <: DescentMethod
    d
    g
end
function init!(M::ConjugateGradientDescent, f, df, x)
    M.g = df(x)
    M.d = -M.g
    return M
end
import Base: step

function step(M::ConjugateGradientDescent, f, df, x)
    d,g=M.d,M.g
    dg =df(x)
    β = max(0, dot(dg, dg-g)/(g⋅g))
    d′ = -dg + β*d
    x′ = line_search(f, x, d′)
    M.d, M.g = d′, dg
    return x′
end        

step (generic function with 10 methods)

In [107]:
M=GradientDescent(0.05)

GradientDescent(0.05)

In [108]:
s=step2(M,f,df,0.1)

0.05024979173609871

In [109]:
s2=step2(M,f,df,s)

0.0003129044934652575